In [3]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

data_path = r'e:\projects\Python\FindingElo\data\Lichess\stockfish\\'
all_features_path = data_path+'all_features\\all_features.pickle'

In [8]:
# Считываем вычисленные характеристики и разделяем
# их на тренировочную и тестовую выборки
X = pd.read_pickle(all_features_path)
X

,0,1,2,3,4,5,6,7,8,9,...,2273,2274,2275,2276,2277,2278,2279,2280,2281,2282
0,5.0,0.0,17.0,0.0,0.0,1.0,1.0,7.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
1,11.0,0.0,12.0,0.0,0.0,1.0,2.0,6.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,7.0,30.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,10.0,0.0,1.0,1.0,0.0,4.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0
4,8.0,0.0,15.0,0.0,1.0,1.0,1.0,4.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27796,9.0,0.0,12.0,0.0,2.0,1.0,0.0,4.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
27797,0.0,0.0,4.0,5.0,2.0,1.0,1.0,7.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
27798,0.0,0.0,6.0,15.0,0.0,1.0,0.0,3.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
27799,0.0,0.0,6.0,27.0,1.0,1.0,0.0,0.0,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
with open(data_path+'data.pickle', 'rb') as f:
    data = pd.read_pickle(f)
data

,ELO_white,ELO_black,Result,ECO,Opening,UTCDate,TimeControl,Termination
0,2126.0,2576.0,1.0,A40,English Defense #2,2013-10-15,bullet,Normal
1,2526.0,2163.0,1.0,A00,Van't Kruijs Opening,2013-10-15,bullet,Normal
2,1767.0,2508.0,1.0,B21,Sicilian Defense: McDonnell Attack,2013-10-17,blitz,Normal
3,2558.0,2006.0,1.0,B14,Caro-Kann Defense: Panov Attack,2013-10-20,bullet,Time forfeit
4,1762.0,2480.0,1.0,B78,"Sicilian Defense: Dragon Variation, Yugoslav A...",2013-10-21,bullet,Normal
...,...,...,...,...,...,...,...,...
27796,2456.0,2304.0,1.0,B07,Rat Defense: Antal Defense,2015-09-30,bullet,Normal
27797,2307.0,2526.0,0.0,B07,Pirc Defense #4,2015-09-30,bullet,Time forfeit
27798,2306.0,2454.0,1.0,A04,Reti Opening #2,2015-09-30,bullet,Time forfeit
27799,2533.0,2301.0,1.0,A00,Hungarian Opening,2015-09-30,bullet,Normal


In [10]:
mean_elos = (data.ELO_white + data.ELO_black) / 2
elo_difference = data.ELO_white - data.ELO_black

In [11]:
X_train_mean, X_test_mean, Y_train_mean, Y_test_mean = train_test_split(X, mean_elos, test_size=0.2, random_state=0)
X_train_diff, X_test_diff, Y_train_diff, Y_test_diff = train_test_split(X, elo_difference, test_size=0.2, random_state=0)

<font size=6>ElasticNet predict mean and elo difference

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV

# Эластичная  модель с итеративной подгонкой вдоль пути регуляризации.
# Предсказываем среднее значение ELO (между белыми и черными)
encv_mean_model = ElasticNetCV()
encv_mean_model.fit(X_train_mean, Y_train_mean)

# а также разницу в ретингах
encv_diff_model = ElasticNetCV()
encv_diff_model.fit(X_train_diff, Y_train_diff)

encv_mean_pred = encv_mean_model.predict(X_test_mean)
encv_diff_pred = encv_diff_model.predict(X_test_diff)
print(mean_absolute_percentage_error(encv_mean_pred, Y_test_mean))
print(mean_absolute_percentage_error(encv_diff_pred, Y_test_diff))

0.0478629578712397
14.79464568698671


<font size =6>RandomForest predict mean and elo difference

In [13]:
# Задаем количество деревьев = 500 и задействуем все ядра процессора
rf_mean = RandomForestRegressor(n_estimators=500, n_jobs=-1)
rf_mean.fit(X_train_mean, Y_train_mean)
rf_mean_pred = rf_mean.predict(X_test_mean)
mean_absolute_percentage_error(rf_mean_pred, Y_test_mean)

0.045130135259089564

In [14]:
rf_diff = RandomForestRegressor(n_estimators=500, n_jobs=-1)
rf_diff.fit(X_train_diff, Y_train_diff)
rf_diff_pred = rf_diff.predict(X_test_diff)
mean_absolute_percentage_error(rf_diff_pred, Y_test_diff)

36.456182554201085

<font size=6>Градиентный бустинг

In [15]:
from sklearn.ensemble import GradientBoostingRegressor 

gbm_mean = GradientBoostingRegressor()
gbm_mean.fit(X_train_mean, Y_train_mean)
gbm_mean_pred = gbm_mean.predict(X_test_mean)
mean_absolute_percentage_error(gbm_mean_pred, Y_test_mean)

0.046644164013150906

In [16]:
gbm_diff = GradientBoostingRegressor()
gbm_diff.fit(X_train_diff, Y_train_diff)
gbm_diff_pred = gbm_mean.predict(X_test_diff)
mean_absolute_percentage_error(gbm_diff_pred, Y_test_diff)

1.0094310892411866

<font size=6>AdaBoost

In [17]:
from sklearn.ensemble import AdaBoostRegressor

ada_mean = AdaBoostRegressor(n_estimators = 25, learning_rate = .05)
ada_mean.fit(X_train_mean, Y_train_mean)
ada_mean_pred = gbm_mean.predict(X_test_mean)
mean_absolute_percentage_error(ada_mean_pred, Y_test_mean)

0.046644164013150906

In [18]:
ada_diff = AdaBoostRegressor(n_estimators = 25, learning_rate = .05)
ada_diff.fit(X_train_diff, Y_train_diff)
ada_diff_pred = gbm_mean.predict(X_test_diff)
mean_absolute_percentage_error(ada_diff_pred, Y_test_diff)

1.0094310892411866